In [ ]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fredapi import Fred
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

class MacroeconomicAnalysis:
    def __init__(self):
        """Initialize the analysis with real-world data"""
        self.data = self.fetch_macro_data()
        self.model_params = self.estimate_ad_as_parameters()
    
    def fetch_macro_data(self):
        """Fetch and process macroeconomic data"""
        # Create sample data (replace with actual FRED API calls)
        dates = pd.date_range(start='2000-01-01', end='2023-12-31', freq='Q')
        np.random.seed(42)
        
        # Generate realistic looking data
        gdp = 100 * np.exp(0.02 * np.arange(len(dates)) / 4 + 
                          0.03 * np.sin(np.arange(len(dates)) / 8) +
                          np.random.normal(0, 0.01, len(dates)).cumsum())
        
        price_level = 100 * np.exp(0.025 * np.arange(len(dates)) / 4 +
                                  np.random.normal(0, 0.005, len(dates)).cumsum())
        
        interest_rate = 2 + 2 * np.sin(np.arange(len(dates)) / 12) + \
                       np.random.normal(0, 0.2, len(dates))
        
        unemployment = 5 + 2 * np.sin(np.arange(len(dates)) / 10) + \
                      np.random.normal(0, 0.3, len(dates))
        
        return pd.DataFrame({
            'Date': dates,
            'Real_GDP': gdp,
            'Price_Level': price_level,
            'Interest_Rate': interest_rate,
            'Unemployment': unemployment
        }).set_index('Date')
    
    def estimate_ad_as_parameters(self):
        """Estimate AD-AS model parameters from the data"""
        # Calculate growth rates and inflation
        self.data['GDP_Growth'] = self.data['Real_GDP'].pct_change() * 100
        self.data['Inflation'] = self.data['Price_Level'].pct_change() * 100
        
        # Estimate potential GDP using HP filter
        from statsmodels.tsa.filters.hp_filter import hpfilter
        self.data['GDP_Trend'], _ = hpfilter(self.data['Real_GDP'], lamb=1600)
        self.data['Output_Gap'] = ((self.data['Real_GDP'] - self.data['GDP_Trend']) / 
                                  self.data['GDP_Trend'] * 100)
        
        # Estimate AD curve (simplified)
        X = self.data['Price_Level'].values.reshape(-1, 1)
        y = self.data['Real_GDP'].values
        reg = LinearRegression().fit(X, y)
        
        # Estimate AS curve (simplified Phillips Curve)
        X_pc = self.data['Output_Gap'].values[1:].reshape(-1, 1)
        y_pc = self.data['Inflation'].values[1:]
        reg_pc = LinearRegression().fit(X_pc, y_pc)
        
        return {
            'ad_intercept': reg.intercept_,
            'ad_slope': -reg.coef_[0],
            'as_slope': reg_pc.coef_[0],
            'natural_rate': self.data['GDP_Trend'].mean()
        }
    
    def plot_historical_analysis(self):
        """Create comprehensive historical analysis plots"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Real GDP and Trend',
                'Price Level and Inflation',
                'Output Gap vs Inflation',
                'AD-AS Dynamics'
            )
        )
        
        # Plot 1: Real GDP and Trend
        fig.add_trace(
            go.Scatter(
                x=self.data.index,
                y=self.data['Real_GDP'],
                name='Real GDP',
                line=dict(color='blue')
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=self.data.index,
                y=self.data['GDP_Trend'],
                name='Potential GDP',
                line=dict(color='red', dash='dash')
            ),
            row=1, col=1
        )
        
        # Plot 2: Price Level and Inflation
        fig.add_trace(
            go.Scatter(
                x=self.data.index,
                y=self.data['Price_Level'],
                name='Price Level',
                line=dict(color='green')
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=self.data.index,
                y=self.data['Inflation'],
                name='Inflation Rate',
                line=dict(color='orange'),
                yaxis='y3'
            ),
            row=1, col=2
        )
        
        # Plot 3: Phillips Curve (Output Gap vs Inflation)
        fig.add_trace(
            go.Scatter(
                x=self.data['Output_Gap'],
                y=self.data['Inflation'],
                mode='markers',
                name='Phillips Curve',
                marker=dict(
                    color=self.data.index.year,
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title='Year')
                )
            ),
            row=2, col=1
        )
        
        # Plot 4: AD-AS Dynamics (Price Level vs Real GDP)
        fig.add_trace(
            go.Scatter(
                x=self.data['Real_GDP'],
                y=self.data['Price_Level'],
                mode='markers',
                name='AD-AS Dynamics',
                marker=dict(
                    color=self.data.index.year,
                    colorscale='Viridis',
                    showscale=False
                )
            ),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=800,
            width=1200,
            showlegend=True,
            title_text="Historical Macroeconomic Analysis"
        )
        
        # Update axes labels
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_xaxes(title_text="Output Gap (%)", row=2, col=1)
        fig.update_xaxes(title_text="Real GDP", row=2, col=2)
        
        fig.update_yaxes(title_text="Real GDP", row=1, col=1)
        fig.update_yaxes(title_text="Price Level", row=1, col=2)
        fig.update_yaxes(title_text="Inflation Rate (%)", row=2, col=1)
        fig.update_yaxes(title_text="Price Level", row=2, col=2)
        
        fig.show()
        
        # Display analysis
        self.display_analysis()
    
    def display_analysis(self):
        """Display comprehensive economic analysis"""
        analysis = f"""
        ### 📊 Macroeconomic Analysis Results
        
        #### 1. Growth and Inflation:
        - Average GDP Growth: {self.data['GDP_Growth'].mean():.2f}%
        - Average Inflation: {self.data['Inflation'].mean():.2f}%
        - GDP Growth Volatility: {self.data['GDP_Growth'].std():.2f}%
        - Inflation Volatility: {self.data['Inflation'].std():.2f}%
        
        #### 2. Model Parameters (Estimated):
        - AD Intercept: {self.model_params['ad_intercept']:.2f}
        - AD Slope: {self.model_params['ad_slope']:.2f}
        - AS Slope (Phillips Curve): {self.model_params['as_slope']:.2f}
        - Natural Rate of Output: {self.model_params['natural_rate']:.2f}
        
        #### 3. Business Cycle Analysis:
        - Current Output Gap: {self.data['Output_Gap'].iloc[-1]:.2f}%
        - Maximum Output Gap: {self.data['Output_Gap'].max():.2f}%
        - Minimum Output Gap: {self.data['Output_Gap'].min():.2f}%
        
        #### 4. Policy Implications:
        - {"Economy appears to be above potential" if self.data['Output_Gap'].iloc[-1] > 0 else "Economy appears to be below potential"}
        - {"Inflationary pressures are building" if self.data['Inflation'].iloc[-1] > self.data['Inflation'].mean() else "Inflationary pressures are subdued"}
        - {"Contractionary policy may be warranted" if self.data['Output_Gap'].iloc[-1] > 1 and self.data['Inflation'].iloc[-1] > 2 else "Expansionary policy may be warranted" if self.data['Output_Gap'].iloc[-1] < -1 else "Current policy stance may be appropriate"}
        """
        
        display(Markdown(analysis))

# Create and run the analysis
macro_analysis = MacroeconomicAnalysis()
macro_analysis.plot_historical_analysis()

In [ ]:
class PolicySimulator:
    def __init__(self, initial_params):
        """Initialize policy simulator with economic parameters"""
        self.params = initial_params
        self.reset_simulation()
    
    def reset_simulation(self):
        """Reset simulation to initial conditions"""
        self.gdp = [self.params['initial_gdp']]
        self.price_level = [self.params['initial_price']]
        self.inflation = [0]
        self.interest_rate = [self.params['initial_rate']]
        self.output_gap = [0]
        self.time = list(range(self.params['simulation_periods']))
    
    def simulate_policy(self, policy_type, magnitude):
        """Simulate the effects of different policy actions"""
        self.reset_simulation()
        
        for t in range(1, self.params['simulation_periods']):
            # Calculate output gap
            potential_gdp = self.params['initial_gdp'] * (1 + self.params['potential_growth'])**(t/4)
            self.output_gap.append((self.gdp[-1] - potential_gdp) / potential_gdp * 100)
            
            # Policy effects
            policy_effect = self.calculate_policy_effect(policy_type, magnitude, t)
            
            # Update variables
            new_gdp = self.update_gdp(t, policy_effect)
            new_price = self.update_price_level(t)
            new_inflation = (new_price - self.price_level[-1]) / self.price_level[-1] * 100
            new_rate = self.update_interest_rate(t, new_inflation)
            
            # Store results
            self.gdp.append(new_gdp)
            self.price_level.append(new_price)
            self.inflation.append(new_inflation)
            self.interest_rate.append(new_rate)
    
    def calculate_policy_effect(self, policy_type, magnitude, t):
        """Calculate the effect of policy on GDP"""
        if policy_type == 'monetary':
            # Monetary policy works with a lag
            return -magnitude * np.exp(-t/4) if t > 2 else 0
        elif policy_type == 'fiscal':
            # Fiscal policy has immediate but temporary effect
            return magnitude * np.exp(-t/8)
        elif policy_type == 'supply_shock':
            # Supply shock has persistent effect
            return -magnitude * (1 - np.exp(-t/12))
        return 0
    
    def update_gdp(self, t, policy_effect):
        """Update GDP based on various factors"""
        trend_growth = self.params['initial_gdp'] * self.params['potential_growth']
        interest_effect = -0.5 * (self.interest_rate[-1] - self.params['neutral_rate'])
        price_effect = -0.3 * (self.inflation[-1] - self.params['inflation_target'])
        
        return self.gdp[-1] * (1 + trend_growth/4 + interest_effect/100 + 
                              price_effect/100 + policy_effect/100)
    
    def update_price_level(self, t):
        """Update price level based on Phillips Curve relationship"""
        expected_inflation = self.params['inflation_target']
        phillips_curve = 0.5 * self.output_gap[-1]
        inflation = expected_inflation + phillips_curve
        
        return self.price_level[-1] * (1 + inflation/100)
    
    def update_interest_rate(self, t, inflation):
        """Update interest rate based on Taylor Rule"""
        taylor_rule = (self.params['neutral_rate'] + 
                      1.5 * (inflation - self.params['inflation_target']) + 
                      0.5 * self.output_gap[-1])
        
        return max(0, taylor_rule)
    
    def plot_simulation_results(self):
        """Plot the simulation results"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Real GDP',
                'Inflation Rate',
                'Output Gap',
                'Interest Rate'
            )
        )
        
        # Plot GDP
        fig.add_trace(
            go.Scatter(
                x=self.time,
                y=self.gdp,
                name='Real GDP',
                line=dict(color='blue')
            ),
            row=1, col=1
        )
        
        # Plot Inflation
        fig.add_trace(
            go.Scatter(
                x=self.time,
                y=self.inflation,
                name='Inflation',
                line=dict(color='red')
            ),
            row=1, col=2
        )
        
        # Plot Output Gap
        fig.add_trace(
            go.Scatter(
                x=self.time,
                y=self.output_gap,
                name='Output Gap',
                line=dict(color='green')
            ),
            row=2, col=1
        )
        
        # Plot Interest Rate
        fig.add_trace(
            go.Scatter(
                x=self.time,
                y=self.interest_rate,
                name='Interest Rate',
                line=dict(color='purple')
            ),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=800,
            width=1200,
            showlegend=True,
            title_text="Policy Simulation Results"
        )
        
        fig.show()
        
        # Display analysis
        self.display_simulation_analysis()
    
    def display_simulation_analysis(self):
        """Display analysis of simulation results"""
        analysis = f"""
        ### 📊 Policy Simulation Analysis
        
        #### 1. Economic Outcomes:
        - Final GDP Growth: {((self.gdp[-1]/self.gdp[0])**(4/len(self.gdp)) - 1)*100:.2f}% (annualized)
        - Average Inflation: {np.mean(self.inflation[1:]):.2f}%
        - Maximum Output Gap: {max(self.output_gap):.2f}%
        - Minimum Output Gap: {min(self.output_gap):.2f}%
        
        #### 2. Policy Effectiveness:
        - Time to Half Output Gap: {self.calculate_half_life('output_gap'):.1f} quarters
        - Time to Target Inflation: {self.calculate_time_to_target():.1f} quarters
        
        #### 3. Policy Trade-offs:
        - Inflation Volatility: {np.std(self.inflation[1:]):.2f}
        - Output Gap Volatility: {np.std(self.output_gap):.2f}
        - Interest Rate Volatility: {np.std(self.interest_rate):.2f}
        
        #### 4. Policy Implications:
        {self.generate_policy_recommendations()}
        """
        
        display(Markdown(analysis))
    
    def calculate_half_life(self, variable):
        """Calculate time to half the initial deviation"""
        series = getattr(self, variable)
        initial = abs(series[0])
        for t, value in enumerate(series):
            if abs(value) <= initial/2:
                return t
        return len(series)
    
    def calculate_time_to_target(self):
        """Calculate time to reach target inflation"""
        target = self.params['inflation_target']
        for t, inf in enumerate(self.inflation):
            if abs(inf - target) <= 0.5:
                return t
        return len(self.inflation)
    
    def generate_policy_recommendations(self):
        """Generate policy recommendations based on simulation results"""
        recommendations = []
        
        # Check inflation
        avg_inflation = np.mean(self.inflation[-4:])
        if avg_inflation > self.params['inflation_target'] + 1:
            recommendations.append("Consider tightening monetary policy to control inflation")
        elif avg_inflation < self.params['inflation_target'] - 1:
            recommendations.append("Consider easing monetary policy to boost inflation")
        
        # Check output gap
        avg_output_gap = np.mean(self.output_gap[-4:])
        if avg_output_gap < -2:
            recommendations.append("Consider expansionary fiscal policy to close negative output gap")
        elif avg_output_gap > 2:
            recommendations.append("Consider contractionary fiscal policy to prevent overheating")
        
        # Check volatility
        if np.std(self.output_gap) > 2:
            recommendations.append("High economic volatility suggests need for automatic stabilizers")
        
        return "- " + "\n- ".join(recommendations)

# Initialize simulator with parameters
initial_params = {
    'initial_gdp': 100,
    'initial_price': 100,
    'initial_rate': 2,
    'potential_growth': 0.02,  # 2% annual
    'neutral_rate': 2,
    'inflation_target': 2,
    'simulation_periods': 20
}

# Create simulator
simulator = PolicySimulator(initial_params)

# Create interactive policy simulation
@interact
def simulate_policy_scenario(
    policy_type=widgets.Dropdown(
        options=['monetary', 'fiscal', 'supply_shock'],
        value='monetary',
        description='Policy Type:'
    ),
    magnitude=widgets.FloatSlider(
        min=-5,
        max=5,
        step=0.5,
        value=1,
        description='Magnitude (%):',
    )
):
    simulator.simulate_policy(policy_type, magnitude)
    simulator.plot_simulation_results()

# 🎯 Advanced Exercises and Case Studies

## 📊 Case Study 1: The Great Recession (2008-2009)

### Background
The 2008 Financial Crisis led to a severe contraction in aggregate demand. Using our AD-AS framework and policy simulator:

1. **Initial Conditions**
   - Set up a negative demand shock of -4% (magnitude = -4)
   - Use the policy simulator to model the economy's response

2. **Policy Response**
   - Simulate the Federal Reserve's response (near-zero interest rates)
   - Model fiscal stimulus (American Recovery and Reinvestment Act)
   - Compare effectiveness of monetary vs. fiscal policy in this scenario

3. **Analysis Questions**
   - Why was monetary policy alone insufficient?
   - How did the zero lower bound constrain policy options?
   - What role did expectations play in the recovery?

## 📈 Case Study 2: Post-COVID Inflation (2021-2023)

### Background
The post-pandemic period saw significant inflation in many economies. Let's analyze this through our framework:

1. **Multiple Shocks**
   - Supply chain disruptions (negative supply shock)
   - Fiscal stimulus (positive demand shock)
   - Labor market changes (structural changes)

2. **Policy Challenges**
   - Use the simulator to model the impact of combined shocks
   - Simulate different policy responses:
     * Aggressive monetary tightening
     * Gradual tightening
     * No policy response

3. **Analysis Questions**
   - Why was inflation more persistent than initially expected?
   - How did supply and demand factors interact?
   - What are the costs of different policy approaches?

## 🏛️ Exercise Set: Policy Analysis

### Exercise 1: Optimal Policy Mix
Using the policy simulator:

1. Consider an economy hit by:
   - Negative supply shock (-2%)
   - Positive demand shock (+3%)
   
2. Find the optimal combination of:
   - Monetary policy adjustment
   - Fiscal policy response
   
3. Compare outcomes under different policy rules:
   - Standard Taylor Rule
   - Modified rules with different weights
   - Discretionary policy

### Exercise 2: Expectation Management
Explore how expectations affect policy effectiveness:

1. Modify the simulator parameters:
   - Change inflation expectations
   - Adjust policy credibility
   
2. Compare outcomes:
   - With anchored expectations
   - With unanchored expectations
   
3. Analyze the role of central bank communication

## 🌍 Real-World Data Analysis Project

### Project Guidelines

1. **Data Collection**
   - Download real GDP and inflation data from FRED
   - Collect information on policy actions
   - Gather expectations data (surveys, market-based)

2. **Analysis Steps**
   ```python
   # Example code structure
   import pandas as pd
   import plotly.express as px
   
   # 1. Load and process data
   def load_economic_data():
       # Your code here
       pass
   
   # 2. Identify shocks
   def identify_economic_shocks(data):
       # Your code here
       pass
   
   # 3. Analyze policy responses
   def analyze_policy_effectiveness(data):
       # Your code here
       pass
   ```

3. **Deliverables**
   - Time series analysis of shocks
   - Policy response evaluation
   - Counterfactual analysis
   - Policy recommendations

## 🎓 Advanced Topics for Further Study

1. **Nonlinear Effects**
   - Zero lower bound on interest rates
   - Capacity constraints
   - Hysteresis effects

2. **Expectations Formation**
   - Rational expectations
   - Adaptive learning
   - Behavioral approaches

3. **International Dimensions**
   - Exchange rate channels
   - International policy spillovers
   - Global supply chains

## 📚 Additional Resources

1. **Academic Papers**
   - Blanchard, O. (2023). "Inflation Dynamics: Dead or Dormant?"
   - Gali, J. (2015). "Monetary Policy, Inflation, and the Business Cycle"

2. **Policy Analysis**
   - Federal Reserve Economic Data (FRED)
   - IMF World Economic Outlook
   - BIS Annual Reports

3. **Online Resources**
   - [FRED Economic Data](https://fred.stlouisfed.org/)
   - [IMF Data](https://www.imf.org/en/Data)
   - [World Bank Open Data](https://data.worldbank.org/)

## 🚀 Next Steps

1. **Research Projects**
   - Choose a historical episode
   - Apply the AD-AS framework
   - Develop policy recommendations

2. **Model Extensions**
   - Add financial sector
   - Incorporate expectations
   - Model open economy effects

3. **Policy Analysis**
   - Compare across countries
   - Evaluate policy frameworks
   - Assess reform proposals

# 📈 The Aggregate Demand - Aggregate Supply (AD-AS) Model: A Dynamic View

The AD-AS model is a cornerstone of macroeconomics, providing a framework to analyze fluctuations in overall economic activity (Real GDP, $Y$) and the aggregate price level ($P$). It helps us understand how shocks to the economy (like changes in government spending, monetary policy, or supply costs) affect output and inflation in the short run and how the economy adjusts back towards its long-run potential over time.

This interactive simulation focuses on the **dynamic adjustment** process. We'll see how output and prices evolve period by period when the economy starts away from its long-run equilibrium, driven by the interplay of aggregate demand and the gradual adjustment of prices described by the aggregate supply relationship.

# 📉 Aggregate Demand (AD)

The Aggregate Demand (AD) curve shows the relationship between the overall price level ($P$) and the total quantity of goods and services demanded in an economy (Real GDP, $Y$). It typically slopes downward.

In this simplified model, we represent the AD curve with a linear equation:

$$Y = a - bP$$

- $Y$: Real GDP (Output)
- $P$: Aggregate Price Level
- $a$: Autonomous spending component (AD Intercept). This captures factors affecting demand other than the price level (e.g., consumer confidence, government spending, investment sentiment, net exports, money supply effects). A higher $a$ shifts the AD curve to the right.
- $b$: Sensitivity of demand to the price level (determines the slope). A larger $b$ means demand is more sensitive to price changes, making the AD curve flatter. This sensitivity arises from effects like:
    - **Wealth Effect:** Higher prices reduce the real value of wealth, dampening consumption.
    - **Interest Rate Effect:** Higher prices can lead to higher interest rates (if money supply is fixed), reducing investment and consumption.
    - **Exchange Rate Effect:** Higher domestic prices can make exports more expensive and imports cheaper, reducing net exports.

Use the sliders for $a$ and $b$ to see how changes in autonomous spending or price sensitivity affect the simulation.

# ⏳ Aggregate Supply (AS) and Price Adjustment Dynamics

The Aggregate Supply (AS) side describes the relationship between the price level ($P$) and the quantity of goods and services firms are willing to supply ($Y$). In the long run, output is determined by the economy's productive capacity (potential output, $Y_{\text{bar}}$), represented by a vertical Long-Run Aggregate Supply (LRAS) curve.

In the short run, however, prices may not adjust instantly. Firms might adjust prices based on the state of the economy relative to its potential. This simulation models a gradual price adjustment process often linked to Phillips Curve concepts: prices tend to rise when output is above potential ($Y > Y_{\text{bar}}$) and fall when output is below potential ($Y < Y_{\text{bar}}$).

The specific dynamic rule used here is:

$$P_t = P_{t-1} + \lambda (Y_{t-1} - Y_{\text{bar}})$$

- $P_t$: Price level in the current period $t$.
- $P_{t-1}$: Price level in the previous period $t-1$.
- $Y_{t-1}$: Output in the previous period $t-1$.
- $Y_{\text{bar}}$: Potential Output (Long-Run Equilibrium Output). This is the level of output sustainable in the long run given the economy's resources and technology.
- $\lambda$: Price adjustment speed parameter. It determines how quickly prices respond to output gaps ($Y - Y_{\text{bar}}$).

In our simulation code, we link the slider parameter $c$ (labeled 'Price Adj. Inertia') inversely to this adjustment speed: $\lambda = 1/c$.
- A **smaller $c$** means a **larger $\lambda$**, implying **faster** price adjustments (a steeper short-run AS relationship conceptually).
- A **larger $c$** means a **smaller $\lambda$**, implying **slower** price adjustments (a flatter short-run AS relationship conceptually).

The simulation starts with an initial price level $P_0$. The interaction between the AD curve and this price adjustment rule drives the economy's path over time.

# 🏁 Conclusion & Interpretation

This simulation demonstrates how the interaction between aggregate demand and a gradual price adjustment mechanism can generate **business cycle dynamics**.

Here's a breakdown of the key takeaways:

* **Shocks:** Changes in the AD parameters ($a$, $b$) or deviations of the initial price level ($P_0$) from the long-run equilibrium price ($P^*$) act as **economic shocks**. These shocks initially move the economy *away* from its potential output ($Y_{\text{bar}}$).

* **Adjustment Mechanism:** The economy doesn't stay shocked forever. It gradually adjusts back towards $Y_{\text{bar}}$ through the following process:
    1.  An **output gap** ($Y_{t-1} - Y_{\text{bar}}$) exists.
    2.  This gap causes the **price level to change** in the next period ($P_t \neq P_{t-1}$) according to the AS dynamics ($\lambda = 1/c$).
    3.  The change in the price level then affects **aggregate demand** in that period ($Y_t = a - bP_t$).
    4.  This continues until the output gap closes and prices stabilize.

* **Adjustment Speed ($c$):** The parameter $c$ (representing price adjustment *inertia*, where $\lambda = 1/c$ is the *speed*) is crucial.
    * A **smaller $c$** (faster adjustment, higher $\lambda$) leads to a quicker return to $Y_{\text{bar}}$.
    * A **larger $c$** (slower adjustment, lower $\lambda$) implies more persistent deviations of output from potential – the business cycle fluctuations last longer.

Experiment with the sliders to explore these dynamics! Observe how different parameter values influence:

* The **path** of output and prices over time.
* The **speed of convergence** back to the long-run equilibrium.

# ⏳ Aggregate Supply (AS) and Price Adjustment Dynamics

The Aggregate Supply (AS) side describes the relationship between the price level ($P$) and the quantity of goods and services firms are willing to supply ($Y$). In the long run, output is determined by the economy's productive capacity (potential output, $Y_{\text{bar}}$), represented by a vertical Long-Run Aggregate Supply (LRAS) curve.

In the short run, however, prices may not adjust instantly. Firms might adjust prices based on the state of the economy relative to its potential. This simulation models a gradual price adjustment process often linked to Phillips Curve concepts: prices tend to rise when output is above potential ($Y > Y_{\text{bar}}$) and fall when output is below potential ($Y < Y_{\text{bar}}$).

The specific dynamic rule used here is:

$$P_t = P_{t-1} + \lambda (Y_{t-1} - Y_{\text{bar}})$$

- $P_t$: Price level in the current period $t$.
- $P_{t-1}$: Price level in the previous period $t-1$.
- $Y_{t-1}$: Output in the previous period $t-1$.
- $Y_{\text{bar}}$: Potential Output (Long-Run Equilibrium Output). This is the level of output sustainable in the long run given the economy's resources and technology.
- $\lambda$: Price adjustment speed parameter. It determines how quickly prices respond to output gaps ($Y - Y_{\text{bar}}$).

In our simulation code, we link the slider parameter $c$ (labeled 'Price Adj. Inertia') inversely to this adjustment speed: $\lambda = 1/c$.
- A **smaller $c$** means a **larger $\lambda$**, implying **faster** price adjustments (a steeper short-run AS relationship conceptually).
- A **larger $c$** means a **smaller $\lambda$**, implying **slower** price adjustments (a flatter short-run AS relationship conceptually).

The simulation starts with an initial price level $P_0$. The interaction between the AD curve and this price adjustment rule drives the economy's path over time.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider # Use IntSlider for T
from IPython.display import display, Markdown
import warnings # To suppress potential minor warnings if needed

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def ad_as_dynamic_sim(a=100, b=10, Y_bar=100, c=5, P0=1.0, T=15):
    """
    Simulates the dynamic adjustment of Output (Y) and Price Level (P)
    in a simple AD-AS framework over T periods.

    Args:
        a (float): AD Intercept (autonomous spending component).
        b (float): AD Slope parameter (sensitivity of Y to P). Must be > 0.
        Y_bar (float): Potential Output (Long-Run Equilibrium Y).
        c (float): Inverse of price adjustment speed (lambda = 1/c). Must be > 0.
                   Smaller c means faster price adjustment.
        P0 (float): Initial Price Level P(0).
        T (int): Time Horizon (number of periods to simulate).
    """
    # Input validation and ensuring T is integer
    T = int(T)
    b = max(b, 1e-6) # Ensure b is positive
    c = max(c, 1e-6) # Ensure c is positive
    lambda_adjust = 1.0 / c # Price adjustment speed

    # Arrays to store results
    Y = np.zeros(T)
    P = np.zeros(T)

    # --- Initial Period (t=0) ---
    P[0] = P0
    # Calculate initial Y based on initial P and AD curve
    Y[0] = a - b * P[0]

    # --- Simulation Loop (t=1 to T-1) ---
    for t in range(1, T):
        # Price level adjusts based on the *previous* period's output gap
        P[t] = P[t-1] + lambda_adjust * (Y[t-1] - Y_bar)
        P[t] = max(P[t], 1e-6) # Ensure price level doesn't go non-positive

        # Output is determined by the AD curve given the *new* price level P[t]
        Y[t] = a - b * P[t]

    # --- Plotting ---
    fig, axes = plt.subplots(1, 3, figsize=(18, 5.5)) # Increased figure width for 3 plots

    # Plot 1: Output Over Time
    axes[0].plot(range(T), Y, marker='o', linestyle='-', label='Output Y(t)', color='blue', markersize=5)
    axes[0].axhline(Y_bar, linestyle='--', color='gray', label=f'Potential Output Y_bar = {Y_bar:.1f}')
    axes[0].set_title("Output (Y) Over Time")
    axes[0].set_xlabel("Time Period (t)")
    axes[0].set_ylabel("Output (Y)")
    axes[0].grid(True, alpha=0.6)
    axes[0].legend()
    axes[0].set_ylim(min(Y.min(), Y_bar) * 0.95, max(Y.max(), Y_bar) * 1.05) # Dynamic Y limits

    # Plot 2: Price Level Over Time
    axes[1].plot(range(T), P, marker='o', linestyle='-', color='darkgreen', label='Price Level P(t)', markersize=5)
    # Calculate approximate long-run equilibrium price P_star
    P_star = (a - Y_bar) / b if b > 1e-6 else P0 # P where AD intersects LRAS (Y=Y_bar)
    axes[1].axhline(P_star, linestyle='--', color='gray', label=f'Long-Run Price P* ≈ {P_star:.2f}')
    axes[1].set_title("Price Level (P) Over Time")
    axes[1].set_xlabel("Time Period (t)")
    axes[1].set_ylabel("Price Level (P)")
    axes[1].grid(True, alpha=0.6)
    axes[1].legend()
    axes[1].set_ylim(P.min() * 0.95, max(P.max(), P_star) * 1.05) # Dynamic P limits

    # Plot 3: Path in AD-AS (Y-P) Space
    # Plot LRAS
    axes[2].axvline(Y_bar, linestyle='--', color='gray', label=f'LRAS (Y_bar = {Y_bar:.1f})')
    # Plot AD Curve
    p_vals_ad = np.linspace(P.min() * 0.9, max(P.max(), P_star) * 1.1, 100)
    y_vals_ad = a - b * p_vals_ad
    axes[2].plot(y_vals_ad, p_vals_ad, color='skyblue', label=f'AD: Y = {a:.0f} - {b:.1f}P')
    # Plot the dynamic path
    axes[2].plot(Y, P, marker='.', linestyle='-', color='red', label='Dynamic Path (Y(t), P(t))')
    axes[2].scatter(Y[0], P[0], color='red', s=100, zorder=5, label=f'Start (Y₀, P₀)')
    axes[2].scatter(Y[-1], P[-1], color='black', marker='X', s=100, zorder=5, label=f'End (Y_{T-1}, P_{T-1})')
    axes[2].set_title("Path in Output-Price Space")
    axes[2].set_xlabel("Output (Y)")
    axes[2].set_ylabel("Price Level (P)")
    axes[2].grid(True, alpha=0.6)
    axes[2].legend(fontsize='small')
    # Set limits based on simulation range and potential output/price
    axes[2].set_xlim(min(Y.min(), Y_bar) * 0.95, max(Y.max(), Y_bar) * 1.05)
    axes[2].set_ylim(P.min() * 0.95, max(P.max(), P_star) * 1.05)


    fig.suptitle("Dynamic AD–AS Adjustment Simulation", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

    # --- Display Final Values and Equilibrium Info ---
    results_md = f"""
    ### 📊 Simulation Results (After T={T} periods)

    * **Final Output Y({T-1}):** {Y[-1]:.2f} (Potential Y_bar = {Y_bar:.1f})
    * **Final Price Level P({T-1}):** {P[-1]:.2f}
    * **Long-Run Equilibrium:** (Y*, P*) ≈ ({Y_bar:.1f}, {P_star:.2f})
        * *Note: The economy may not fully reach equilibrium within T periods.*
    * **Price Adjustment Speed (λ = 1/c):** {lambda_adjust:.3f} (using c={c:.1f})
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'} # Allow longer descriptions
interact(
    ad_as_dynamic_sim,
    a=FloatSlider(value=100, min=80, max=150, step=5, description='AD Intercept (a):', style=style),
    b=FloatSlider(value=10, min=1, max=30, step=1, description='AD Slope Param (b):', style=style),
    Y_bar=FloatSlider(value=100, min=80, max=120, step=1, description='Potential Output (Y_bar):', style=style),
    c=FloatSlider(value=5, min=0.5, max=20, step=0.5, description='Price Adj. Inertia (c=1/λ):', style=style), # Changed description
    P0=FloatSlider(value=1.0, min=0.5, max=2.0, step=0.05, description='Initial Price Level (P0):', style=style),
    T=IntSlider(value=15, min=5, max=50, step=1, description='Time Horizon (T):', style=style, readout_format='d'), # Changed to IntSlider
);


interactive(children=(FloatSlider(value=100.0, description='AD Intercept (a):', max=150.0, min=80.0, step=5.0,…

# 📝 Guided Student Exercise: Explore AD-AS Dynamics
Let's apply your knowledge and intuition!

1. **Suppose the economy starts with $a=120$, $b=10$, $Y_{\text{bar}}=100$, $c=5$, and $P_0=1.0$.**
    - What is the long-run equilibrium price level $P^*$?
    - Use the interactive simulation to observe how output and prices adjust over time. How many periods does it take for output to get close to $Y_{\text{bar}}$?

2. **Shock the Economy!**
    - Increase $a$ (autonomous spending) by 20 units. What happens to output and prices in the short run and long run?
    - Now, try decreasing $a$ by 20 units. How does the adjustment path differ?

3. **Experiment with Price Adjustment Speed ($c$):**
    - Set $c$ to a very small value (fast adjustment). What do you observe?
    - Set $c$ to a large value (slow adjustment). How does the economy's path change?

4. **Challenge:**
    - Try starting with $P_0$ far from equilibrium (e.g., $P_0=0.5$ or $P_0=2.0$). How does the economy adjust?

---
# 🌍 Real-World Data Extension: U.S. Output and Price Level
Let's connect the model to real-world data. We'll plot U.S. Real GDP and the Consumer Price Index (CPI) over time using FRED data.

```python
import pandas as pd
import plotly.graph_objects as go
gdp_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDPC1'
cpi_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL'
gdp = pd.read_csv(gdp_url)
cpi = pd.read_csv(cpi_url)
gdp['DATE'] = pd.to_datetime(gdp['DATE'])
cpi['DATE'] = pd.to_datetime(cpi['DATE'])
fig = go.Figure()
fig.add_trace(go.Scatter(x=gdp['DATE'], y=gdp['GDPC1'], name='Real GDP (Billions, 2012$)', yaxis='y1'))
fig.add_trace(go.Scatter(x=cpi['DATE'], y=cpi['CPIAUCSL'], name='CPI (Index, 1982-84=100)', yaxis='y2'))
fig.update_layout(title='U.S. Real GDP and Consumer Price Index', xaxis_title='Year', yaxis=dict(title='Real GDP', side='left'), yaxis2=dict(title='CPI', overlaying='y', side='right'))
fig.show()
```

- What patterns do you notice in output and prices during recessions and recoveries?
- How do these patterns relate to the AD-AS model's predictions?

---
# 📚 Further Reading & Resources
- [Mankiw, N. G. (2021). *Macroeconomics* (11th Edition), Chapter 13: Aggregate Demand and Aggregate Supply.](https://www.macmillanlearning.com/college/us/product/Macroeconomics/p/1319243584)
- [FRED: Real GDP (GDPC1)](https://fred.stlouisfed.org/series/GDPC1)
- [FRED: Consumer Price Index (CPIAUCSL)](https://fred.stlouisfed.org/series/CPIAUCSL)
- [Marginal Revolution University: AD-AS Model](https://mru.org/courses/principles-economics-macroeconomics/aggregate-demand-aggregate-supply)

---
# 🎨 Tips for Visual Exploration
- Use the interactive simulation to try different shocks and adjustment speeds.
- Discuss with classmates: What real-world events (e.g., oil shocks, policy changes) might correspond to shifts in $a$ or $b$?
- How might the model be extended to include expectations or supply shocks?

---
# 🚀 Next Steps
- Continue to the next notebook to explore monetary and fiscal policy, expectations, and the Phillips Curve!

# 📝 Guided Student Exercise: Aggregate Demand and Supply

1. **Scenario Analysis:**
    - Suppose the economy experiences a positive demand shock. How does this affect the AD curve? What happens to output and prices in the short run?
    - Use the interactive plot to simulate a supply shock. How does this differ from a demand shock?

2. **Experiment:**
    - Adjust the slope of the AS curve. What happens to the economy's adjustment path?
    - Explore the effects of a simultaneous demand and supply shock.

3. **Challenge:**
    - Analyze the role of expectations in the adjustment process. How does anchoring inflation expectations affect the AS curve?

---
# 🌍 Real-World Data Extension: Inflation and Output Gaps

```python
import pandas as pd
import plotly.express as px
output_gap_url = 'https://example.com/output_gap.csv'
inflation_url = 'https://example.com/inflation.csv'
output_gap = pd.read_csv(output_gap_url)
inflation = pd.read_csv(inflation_url)
output_gap['DATE'] = pd.to_datetime(output_gap['DATE'])
inflation['DATE'] = pd.to_datetime(inflation['DATE'])
fig = px.line(output_gap, x='DATE', y='GAP', title='Output Gap Over Time')
fig.add_scatter(x=inflation['DATE'], y=inflation['RATE'], mode='lines', name='Inflation Rate')
fig.show()
```

- How do output gaps correlate with inflation over time?
- Can you identify periods of stagflation or overheating?

---
# 📚 Further Reading & Resources
- [Blanchard, O. (2021). *Macroeconomics* (8th Edition), Chapter 7: The AD-AS Model.](https://www.pearson.com)
- [FRED: Output Gap](https://fred.stlouisfed.org/series/OUTPUTGAP)
- [FRED: Inflation Rate](https://fred.stlouisfed.org/series/CPIAUCSL)

---
# 🎨 Tips for Visual Exploration
- Use the interactive plot to simulate different shocks.
- Discuss with classmates: What policies can stabilize the economy during shocks?

---
# 🚀 Next Steps
- Continue to the next notebook to explore IS-LM models and monetary policy!